In [ ]:
import glob as glob

import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def read_vars(fName, vName, level_index):
    ds = xr.open_dataset(fName, decode_times=False)
    var = ds[vName].isel(depth=level_index)
    return var

In [ ]:
def plot_var(out_dir, MON, var, varName, varUnits, cMin, cMax, cMap):
    WOA_ver = 'WOA 2023'
    
    fig=plt.figure(figsize=(8, 6), clear=True, dpi=60)
    ax = fig.add_subplot(111, projection=ccrs.Robinson())
    im=ax.pcolormesh(var['lon'], var['lat'], var.squeeze(),
                     vmin=cMin, vmax=cMax, cmap=cMap,
                     transform=ccrs.PlateCarree())
    
    cbar=fig.colorbar(im, ax=ax, orientation='horizontal')
    ax.set_title(f'{WOA_ver} {MON}')
    cbar.ax.set_title(f'{varName} [{varUnits}]')
    figName = out_dir + varName+'_'+MON+'.png'
    plt.savefig(figName)
    print(f'Saved {figName}', bbox_inches='tight')
    plt.close('all')

In [ ]:
varUnits={
    'Temp':'degC',
    'Salt':'PSU'}

varCbar={
    'Temp':[-2, 32, 'gist_ncar'],
    'Salt':[32, 40, 'nipy_spectral']}

## Set inputs:
- Path to data.
- Month of dataset.
- Variable names.
- Depth (index).

## Outputs:
- Output path.

In [ ]:
res = "025"
fPref, fSuff = ["woa23_decav91C0_", "04"]

monStr = "01"
varStr = ["t", "s"]

# set (3-d) variable names and level
vNames = ["t_an", "s_an"]
level_index = 0 # level-1 (aka "k-index")

data_rootPath = "/scratch3/NCEPDEV/climate/Jiande.Wang/OBS-data/WOA2023/" + res+"/"

# path where figures will be saved
out_dir = "/home/Santha.Akella/for_folks/JiandeW/Sep2025/figs/WOA/"

## Read data

In [ ]:
fName_temp = sorted(glob.glob(data_rootPath + fPref + varStr[0] + monStr + "_" + fSuff +".nc"))[0]
print(f"Read temperature from\t{fName_temp}")
t_an = read_vars(fName_temp, vNames[0], level_index)

fName_salt = sorted(glob.glob(data_rootPath + fPref + varStr[1] + monStr + "_" + fSuff +".nc"))[0]
print(f"Read salinity from\t{fName_salt}")
s_an = read_vars(fName_salt, vNames[1], level_index)

## Plot

In [ ]:
# Temperature - strictly it is in-situ, for "SST" it will pass!
plot_var(out_dir, monStr, t_an, vNames[0], 
         varUnits['Temp'], varCbar['Temp'][0], varCbar['Temp'][1], varCbar['Temp'][2])

# Salinity - strictly it needs to be converted to absolute salinity, but for 'SSS" it will pass!
plot_var(out_dir, monStr, s_an, vNames[1], 
         varUnits['Salt'], varCbar['Salt'][0], varCbar['Salt'][1], varCbar['Salt'][2])